In [17]:
import json
import numpy as np
np.random.seed(0)

from bokeh.io import curdoc
from bokeh.layouts import widgetbox, row, column, layout
from bokeh.models import ColumnDataSource, Select, Slider
from bokeh.palettes import Spectral6
from bokeh.io import output_notebook, show
from bokeh.plotting import figure, output_file, show
from sklearn import tree

from sklearn import neighbors, datasets
from sklearn.neighbors import NearestNeighbors
from sklearn import cluster, datasets
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import xlrd
from sklearn.cluster import KMeans
from bokeh.plotting import figure
from bokeh.models import HoverTool
from bokeh.models import BoxZoomTool
from bokeh.models import WheelZoomTool
from bokeh.models import PanTool

In [18]:
file  = open("ranking.json", "r")
text = file.read()

file = open("metaldata.json", "r")
metal = file.read()
#print(text)

In [19]:
parsed_text = json.loads(text)
metal_data = json.loads(metal)

In [20]:
print(parsed_text['objects'][0])
print(len(parsed_text['objects']))

{'googleBooksRank': -1, 'happs': 8.5, 'lyricsRank': 1728, 'newYorkTimesRank': -1, 'rank': 1, 'stdDev': 0.9313, 'text': 'laughter', 'twitterRank': 3600, 'word': 'laughter'}
10222


In [21]:
def stemStepA(word):
    vowels = ['a', 'e', 'i',  'o', 'u', 'y']
    
    if word[len(word)-3:] == "ied" or word[len(word)-3:] == "ies":
        return word[:-2]
    
    
    if word[len(word)-4:] == "sses":
        if len(word) >= 4:
            word = word[:len(word)-4] + "ss"
        return word
    
    
    if word[len(word)-1:] == "s":
        if word[len(word)-2:-1] not in vowels:
            word = word[:-1]    
        return word
    
    
    return word    

In [22]:
def firstNonVowelIndex(word):
    
    vowels = ['a', 'e', 'i',  'o', 'u', 'y']
    
    index = -1
    
    for i in range(0, len(word)):
        if word[i] not in vowels and index == -1:
            index = i
            
    #print("index is ", index)
    return index

In [23]:
def rootHasVowel(word, endingIndex):
    
    vowels = ['a', 'e', 'i',  'o', 'u', 'y']
    
    temp = word[:-endingIndex]
    
    for v in vowels:
        if v in temp:
            return True
         
    return False

In [24]:
def stemStepB(word):
    vowels = ['a', 'e', 'i',  'o', 'u', 'y']
    
    #print(word, "got past 0")
    
    if len(word) >= 4:
        if word[len(word)-3:] == "eed":
            if word[len(word)-4:-3] not in vowels:
                    if len(word)-4 == firstNonVowelIndex(word):
                        return word
                    else:
                        return word[:-3] + "ee"
                    
                    
                    
                    
    #print(word, "got past 1")
        
    if len(word) >= 6:
        if word[len(word)-5:] == "eedly":
            if word[len(word)-6:-5] not in vowels:
                if len(word)-6 == firstNonVowelIndex(word):
                    return word
                else:
                    return word[:-5] + "ee"
        
        
        
        
    #print(word, "got past 2")
    
    if word[len(word)-2:] == "ed":
        if rootHasVowel(word, 2) == True:
            word = word[:-2]
        
        if word[len(word)-2:] == "at" or word[len(word)-2:] == "bl" or word[len(word)-2:] == "az":
            word = word + "e"
            
        if word[len(word)-2:-1] == word[len(word)-1:]:
            if word[len(word)-2:] != "ll" and word[len(word)-2:] != "ss" and word[len(word)-2:] != "zz":                                  
                word = word[:-1]
            
        return word

    
    
    
    #print(word, "got past 3")
    
    if word[len(word)-4:] == "edly":
        if rootHasVowel(word, 4) == True:
            word = word[:-4]
        
        if word[len(word)-2:] == "at" or word[len(word)-2:] == "bl" or word[len(word)-2:] == "az":
            word = word + "e"
            
        if word[len(word)-2:-1] == word[len(word)-1:]:
            if word[len(word)-2:] != "ll" and word[len(word)-2:] != "ss" and word[len(word)-2:] != "zz":                                  
                word = word[:-1]
            
        return word
    
    
    
    
    #print(word, "got past 4")
    
    if word[len(word)-3:] == "ing":
        if rootHasVowel(word, 3) == True:
            word = word[:-3]
            
        if word[len(word)-2:] == "at" or word[len(word)-2:] == "bl" or word[len(word)-2:] == "az":
            word = word + "e"
            
        if word[len(word)-2:-1] == word[len(word)-1:]:
            if word[len(word)-2:] != "ll" and word[len(word)-2:] != "ss" and word[len(word)-2:] != "zz":                                  
                word = word[:-1]
            
            
        return word
    
    
    
    
    #print(word, "got past 5")
    
    if word[len(word)-5:] == "ingly":
        if rootHasVowel(word, 5) == True:
            word = word[:-5]
            
        if word[len(word)-2:] == "at" or word[len(word)-2:] == "bl" or word[len(word)-2:] == "az":
            word = word + "e"
            
        if word[len(word)-2:-1] == word[len(word)-1:]:
            if word[len(word)-2:] != "ll" and word[len(word)-2:] != "ss" and word[len(word)-2:] != "zz":                                  
                word = word[:-1]
            
        return word
        
    return word

In [25]:
def stem(word):
    word = stemStepA(word)
    word = stemStepB(word)
    return word

In [26]:
def findWords(metal_data):
    uniqueWords = []

    for band in metal_data.keys():
        for album in metal_data[band].keys():
            for song in metal_data[band][album].keys():
                lyrics = metal_data[band][album][song]
                wordsInSong = lyrics.split(" ")
                for w in wordsInSong:
                    if w != '':
                        if w not in uniqueWords:
                            uniqueWords.append(stem(w.lower()))
            
    return uniqueWords

In [27]:
def getGoogleAndTwitterRanks(metal_data, parsed_text):
    uniqueWords = findWords(metal_data)
    word_google_twitter = {}
    
    for word in uniqueWords:
        for i in range(0, len(parsed_text['objects'])):
            if parsed_text['objects'][i]['word'] == word:
                if word not in word_google_twitter.keys():
                    word_google_twitter[word] = (parsed_text['objects'][i]['googleBooksRank'], parsed_text['objects'][i]['twitterRank'])                  
                
    
    return word_google_twitter

In [28]:
word_google_twitter = getGoogleAndTwitterRanks(metal_data, parsed_text)

for word in word_google_twitter.keys():
    print("word = ", word)
    print("Google Rank = ", word_google_twitter[word][0])
    print("Twitter Rank = ", word_google_twitter[word][1])

word =  every
Google Rank =  151
Twitter Rank =  244
word =  step
Google Rank =  899
Twitter Rank =  1072
word =  you
Google Rank =  34
Twitter Rank =  9
word =  take
Google Rank =  165
Twitter Rank =  145
word =  is
Google Rank =  7
Twitter Rank =  7
word =  to
Google Rank =  4
Twitter Rank =  3
word =  the
Google Rank =  1
Twitter Rank =  1
word =  end
Google Rank =  198
Twitter Rank =  356
word =  human
Google Rank =  218
Twitter Rank =  1207
word =  slaves
Google Rank =  2701
Twitter Rank =  -1
word =  mother
Google Rank =  437
Twitter Rank =  626
word =  nature
Google Rank =  243
Twitter Rank =  1646
word =  so
Google Rank =  47
Twitter Rank =  23
word =  far
Google Rank =  211
Twitter Rank =  442
word =  away
Google Rank =  272
Twitter Rank =  292
word =  perfect
Google Rank =  1411
Twitter Rank =  642
word =  race
Google Rank =  1197
Twitter Rank =  1413
word =  dream
Google Rank =  2076
Twitter Rank =  409
word =  right
Google Rank =  154
Twitter Rank =  112
word =  i
Google Ra

word =  spin
Google Rank =  -1
Twitter Rank =  3779
word =  round
Google Rank =  856
Twitter Rank =  1063
word =  conscious
Google Rank =  2350
Twitter Rank =  -1
word =  planet
Google Rank =  -1
Twitter Rank =  1779
word =  axis
Google Rank =  2698
Twitter Rank =  -1
word =  holes
Google Rank =  4410
Twitter Rank =  -1
word =  heaven
Google Rank =  1615
Twitter Rank =  1294
word =  wonder
Google Rank =  2309
Twitter Rank =  344
word =  matter
Google Rank =  352
Twitter Rank =  887
word =  can
Google Rank =  51
Twitter Rank =  48
word =  disappear
Google Rank =  -1
Twitter Rank =  -1
word =  door
Google Rank =  765
Twitter Rank =  498
word =  wall
Google Rank =  951
Twitter Rank =  938
word =  reach
Google Rank =  1334
Twitter Rank =  1827
word =  space
Google Rank =  614
Twitter Rank =  571
word =  mount
Google Rank =  3468
Twitter Rank =  -1
word =  nonsense
Google Rank =  -1
Twitter Rank =  3862
word =  simplicity
Google Rank =  4809
Twitter Rank =  -1
word =  touch
Google Rank =  1

Twitter Rank =  1835
word =  cloud
Google Rank =  4103
Twitter Rank =  2978
word =  colour
Google Rank =  2479
Twitter Rank =  3965
word =  slide
Google Rank =  -1
Twitter Rank =  4241
word =  horizon
Google Rank =  -1
Twitter Rank =  -1
word =  frontier
Google Rank =  3695
Twitter Rank =  -1
word =  gone
Google Rank =  960
Twitter Rank =  541
word =  realm
Google Rank =  3957
Twitter Rank =  -1
word =  shut
Google Rank =  3491
Twitter Rank =  1123
word =  slowly
Google Rank =  1685
Twitter Rank =  1837
word =  year
Google Rank =  173
Twitter Rank =  156
word =  thought
Google Rank =  212
Twitter Rank =  161
word =  their
Google Rank =  33
Twitter Rank =  171
word =  moan
Google Rank =  -1
Twitter Rank =  -1
word =  long
Google Rank =  129
Twitter Rank =  191
word =  breath
Google Rank =  2872
Twitter Rank =  2334
word =  unit
Google Rank =  1027
Twitter Rank =  2378
word =  center
Google Rank =  878
Twitter Rank =  1165
word =  greater
Google Rank =  456
Twitter Rank =  3772
word =  s

Google Rank =  689
Twitter Rank =  1191
word =  number
Google Rank =  150
Twitter Rank =  779
word =  board
Google Rank =  728
Twitter Rank =  845
word =  amount
Google Rank =  550
Twitter Rank =  2140
word =  you've
Google Rank =  -1
Twitter Rank =  -1
word =  taught
Google Rank =  1831
Twitter Rank =  3674
word =  devil
Google Rank =  4247
Twitter Rank =  1948
word =  isn't
Google Rank =  -1
Twitter Rank =  -1
word =  closer
Google Rank =  2799
Twitter Rank =  2391
word =  message
Google Rank =  1667
Twitter Rank =  983
word =  goes
Google Rank =  1278
Twitter Rank =  515
word =  key
Google Rank =  1101
Twitter Rank =  746
word =  failure
Google Rank =  1071
Twitter Rank =  1764
word =  waste
Google Rank =  2485
Twitter Rank =  1148
word =  burnt
Google Rank =  -1
Twitter Rank =  -1
word =  essence
Google Rank =  3357
Twitter Rank =  -1
word =  wake
Google Rank =  -1
Twitter Rank =  725
word =  absolute
Google Rank =  1720
Twitter Rank =  4229
word =  worship
Google Rank =  2506
Twit

Google Rank =  715
Twitter Rank =  216
word =  beauty
Google Rank =  1611
Twitter Rank =  1032
word =  chorus
Google Rank =  -1
Twitter Rank =  -1
word =  south
Google Rank =  370
Twitter Rank =  833
word =  stain
Google Rank =  -1
Twitter Rank =  -1
word =  fought
Google Rank =  3193
Twitter Rank =  -1
word =  mercy
Google Rank =  3765
Twitter Rank =  -1
word =  stranger
Google Rank =  4422
Twitter Rank =  4728
word =  temptation
Google Rank =  -1
Twitter Rank =  -1
word =  sinner
Google Rank =  -1
Twitter Rank =  -1
word =  clue
Google Rank =  -1
Twitter Rank =  3648
word =  store
Google Rank =  2151
Twitter Rank =  654
word =  pretend
Google Rank =  -1
Twitter Rank =  3680
word =  correction
Google Rank =  -1
Twitter Rank =  -1
word =  prophet
Google Rank =  4465
Twitter Rank =  -1
word =  enemy
Google Rank =  1361
Twitter Rank =  4424
word =  cause
Google Rank =  444
Twitter Rank =  417
word =  book
Google Rank =  278
Twitter Rank =  229
word =  naive
Google Rank =  -1
Twitter Rank

In [29]:
for dataPoint in parsed_text['objects']:
    if dataPoint['word'] == "death":
        print(dataPoint)

{'googleBooksRank': 307, 'happs': 1.54, 'lyricsRank': 433, 'newYorkTimesRank': 373, 'rank': 10217, 'stdDev': 1.2811, 'text': 'death', 'twitterRank': 509, 'word': 'death'}


In [30]:
def buildScatterPlot(word_google_twitter):
    googleRanks = []
    twitterRanks = []
    
    for word in word_google_twitter.keys():
        googleRanks.append(word_google_twitter[word][0])
        twitterRanks.append(word_google_twitter[word][1])

    p = figure(plot_width=1000, plot_height=1000)
    p.outline_line_width = 7
    p.outline_line_alpha = 0.3
    p.outline_line_color = "navy"
    p.circle(googleRanks, twitterRanks, size=3)
    return p

In [66]:
def createPair(word_google_twitter):
    xyValPairs = []
    
    for word in word_google_twitter.keys():
        onePair = []
        googleRank = int(word_google_twitter[word][0])
        twitterRank =int(word_google_twitter[word][1])
        onePair.append(googleRank)
        onePair.append(twitterRank)
        xyValPairs.append(onePair)
        
        
        
    return xyValPairs

In [83]:
def buildKMeansScatterPlot(word_google_twitter, kValue):
    X = createPair(word_google_twitter)
    print(type(X[0][1]))
    print(len(X))
    
    kmeans = KMeans(n_clusters=kValue, random_state=0).fit(X)
    print("kmeans.labels_ = ",kmeans.labels_)
    print("len(kmeans.labels_) = ", len(kmeans.labels_))
    
    hover = HoverTool(
        tooltips=[
            ("index", "$word[word.keys()[index]]"),
            ("googleRank", "$x"),
            ("twitterRank", "$y")
        ]
    )

    print("making figure")
    p = figure(plot_width=600, plot_height=600, title= "GoogleRank vs TwitterRank",  tools=[hover, PanTool(), BoxZoomTool(), WheelZoomTool()])                    

    print("making points")
    for i in range(0, len(X)):
        if kmeans.labels_[i] == 0:
            print("did for 0 at ", i)
            p.circle(X[i][0], X[i][1], size=3, line_color="green", fill_color="green", fill_alpha=0.5)     
        elif kmeans.labels_[i] == 1:
            print("did for 1 at ", i)
            p.triangle(X[i][0], X[i][1], size=3, line_color="blue", fill_color="blue", fill_alpha=0.5)
        elif kmeans.labels_[i] == 2:
            print("did for 2 at ", i)
            p.square(X[i][0], X[i][1], size=3, line_color="red", fill_color="red", fill_alpha=0.5)
        elif kmeans.labels_[i] == 3:
            print("did for 3 at ", i)
            p.circle(X[i][0], X[i][1], size=3, line_color="blue", fill_color="blue", fill_alpha=0.5)     
        elif kmeans.labels_[i] == 4:
            print("did for 4 at ", i)
            p.triangle(X[i][0], X[i][1], size=3, line_color="red", fill_color="red", fill_alpha=0.5)
        elif kmeans.labels_[i] == 5:
            print("did for 5 at ", i)
            p.square(X[i][0], X[i][1], size=3, line_color="green", fill_color="green", fill_alpha=0.5)
        elif kmeans.labels_[i] == 6:
            print("did for 6 at ", i)
            p.circle(X[i][0], X[i][1], size=3, line_color="red", fill_color="red", fill_alpha=0.5)     
        elif kmeans.labels_[i] == 7:
            print("did for 7 at ", i)
            p.triangle(X[i][0], X[i][1], size=3, line_color="green", fill_color="green", fill_alpha=0.5)
        elif kmeans.labels_[i] == 8:
            print("did for 8 at ", i)
            p.square(X[i][0], X[i][1], size=3, line_color="blue", fill_color="blue", fill_alpha=0.5)
        elif kmeans.labels_[i] == 9:
            print("did for 9 at ", i)
            p.circle(X[i][0], X[i][1], size=3, line_color="black", fill_color="white", fill_alpha=0.5)     
        elif kmeans.labels_[i] == 10:
            print("did for 10 at ", i)
            p.triangle(X[i][0], X[i][1], size=3, line_color="black", fill_color="white", fill_alpha=0.5)
    
    p.xaxis.axis_label = "Google Ranks"
    p.yaxis.axis_label = "Twitter Ranks"
    
    print("showing plot")
    return p

In [86]:
scatterPlot = buildScatterPlot(word_google_twitter)
show(scatterPlot)

In [88]:
kMeansScatterPlot = buildKMeansScatterPlot(word_google_twitter, 2)

<class 'int'>
1549
kmeans.labels_ =  [1 1 1 ..., 1 1 0]
len(kmeans.labels_) =  1549
making figure
making points
did for 1 at  0
did for 1 at  1
did for 1 at  2
did for 1 at  3
did for 1 at  4
did for 1 at  5
did for 1 at  6
did for 1 at  7
did for 1 at  8
did for 0 at  9
did for 1 at  10
did for 1 at  11
did for 1 at  12
did for 1 at  13
did for 1 at  14
did for 1 at  15
did for 1 at  16
did for 0 at  17
did for 1 at  18
did for 1 at  19
did for 1 at  20
did for 1 at  21
did for 1 at  22
did for 0 at  23
did for 1 at  24
did for 1 at  25
did for 1 at  26
did for 0 at  27
did for 1 at  28
did for 1 at  29
did for 1 at  30
did for 1 at  31
did for 1 at  32
did for 1 at  33
did for 0 at  34
did for 1 at  35
did for 1 at  36
did for 1 at  37
did for 0 at  38
did for 1 at  39
did for 1 at  40
did for 1 at  41
did for 0 at  42
did for 0 at  43
did for 1 at  44
did for 1 at  45
did for 1 at  46
did for 0 at  47
did for 1 at  48
did for 1 at  49
did for 1 at  50
did for 1 at  51
did for 1 at  

did for 1 at  456
did for 1 at  457
did for 1 at  458
did for 0 at  459
did for 1 at  460
did for 1 at  461
did for 0 at  462
did for 1 at  463
did for 1 at  464
did for 1 at  465
did for 1 at  466
did for 1 at  467
did for 0 at  468
did for 0 at  469
did for 1 at  470
did for 1 at  471
did for 1 at  472
did for 0 at  473
did for 1 at  474
did for 1 at  475
did for 1 at  476
did for 0 at  477
did for 0 at  478
did for 1 at  479
did for 1 at  480
did for 0 at  481
did for 1 at  482
did for 1 at  483
did for 0 at  484
did for 1 at  485
did for 1 at  486
did for 1 at  487
did for 1 at  488
did for 1 at  489
did for 1 at  490
did for 1 at  491
did for 0 at  492
did for 1 at  493
did for 0 at  494
did for 1 at  495
did for 0 at  496
did for 1 at  497
did for 0 at  498
did for 1 at  499
did for 1 at  500
did for 1 at  501
did for 1 at  502
did for 1 at  503
did for 1 at  504
did for 1 at  505
did for 1 at  506
did for 1 at  507
did for 1 at  508
did for 1 at  509
did for 1 at  510
did for 0 

did for 1 at  912
did for 1 at  913
did for 1 at  914
did for 0 at  915
did for 1 at  916
did for 1 at  917
did for 1 at  918
did for 1 at  919
did for 1 at  920
did for 0 at  921
did for 1 at  922
did for 1 at  923
did for 1 at  924
did for 1 at  925
did for 1 at  926
did for 0 at  927
did for 0 at  928
did for 0 at  929
did for 0 at  930
did for 1 at  931
did for 1 at  932
did for 1 at  933
did for 1 at  934
did for 1 at  935
did for 0 at  936
did for 1 at  937
did for 0 at  938
did for 1 at  939
did for 0 at  940
did for 0 at  941
did for 1 at  942
did for 1 at  943
did for 0 at  944
did for 0 at  945
did for 0 at  946
did for 0 at  947
did for 1 at  948
did for 1 at  949
did for 1 at  950
did for 1 at  951
did for 1 at  952
did for 1 at  953
did for 0 at  954
did for 0 at  955
did for 0 at  956
did for 0 at  957
did for 1 at  958
did for 0 at  959
did for 0 at  960
did for 1 at  961
did for 1 at  962
did for 0 at  963
did for 1 at  964
did for 1 at  965
did for 0 at  966
did for 0 

did for 0 at  1348
did for 1 at  1349
did for 1 at  1350
did for 0 at  1351
did for 0 at  1352
did for 1 at  1353
did for 1 at  1354
did for 1 at  1355
did for 1 at  1356
did for 1 at  1357
did for 1 at  1358
did for 1 at  1359
did for 1 at  1360
did for 1 at  1361
did for 0 at  1362
did for 1 at  1363
did for 1 at  1364
did for 1 at  1365
did for 1 at  1366
did for 0 at  1367
did for 0 at  1368
did for 1 at  1369
did for 1 at  1370
did for 1 at  1371
did for 0 at  1372
did for 1 at  1373
did for 0 at  1374
did for 1 at  1375
did for 0 at  1376
did for 0 at  1377
did for 1 at  1378
did for 1 at  1379
did for 1 at  1380
did for 1 at  1381
did for 1 at  1382
did for 0 at  1383
did for 0 at  1384
did for 0 at  1385
did for 0 at  1386
did for 1 at  1387
did for 1 at  1388
did for 1 at  1389
did for 1 at  1390
did for 0 at  1391
did for 0 at  1392
did for 1 at  1393
did for 1 at  1394
did for 0 at  1395
did for 1 at  1396
did for 1 at  1397
did for 1 at  1398
did for 1 at  1399
did for 1 at